In [55]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.getcwd())
from src.taxodist import td_utils as utils
from src.taxodist import td_calc
from src.taxodist import tree_parsers

In [94]:
df = pd.read_excel('analysis\local\\resources\pankreas_patienten_matrix_MB.xlsx')
df = df.drop(axis=1,columns='Unnamed: 0')
df = df.set_index(df.columns)
df[df.isna()] = 0.0

expert_matrix = utils.mirrorMatrix(df.to_numpy())
#df_mds_dist = utils.getMDSMatrix(matrix)
#utils.plotDistMatrix(df_mds_dist,df.columns)

In [56]:
df_hd_moltb = pd.read_excel('analysis/resources/pseudo_icd10_hauptdiagnose_moltb.xlsx')
df_pancreas_hd = df_hd_moltb[df_hd_moltb['ICD-10 Hauptdiagnose'].str.contains('C25')]

df_moltb = pd.read_excel('analysis/resources/pseudo_icd10_moltb.xlsx')
df_patient_icd = df_moltb.groupby(df_moltb.PseudoPatNr.name)['ICD'].agg(list).reset_index(name='ICD')
df_pancreas_patient = df_patient_icd[df_patient_icd['PseudoPatNr'].isin(df_pancreas_hd['Pseudonym'])]

pancreas_patient_array = df_pancreas_patient.to_numpy()
pancreas_icd_sets = [set(icd_list) for patient, icd_list in pancreas_patient_array]
# setnames = [str(pseudonym) for pseudonym, icd in pancreas_patient_array]
setnames = ['patient ' + str(i) for i, icd in enumerate(pancreas_patient_array)]

tree = tree_parsers.getICD10GMTree(version='2021')
td = td_calc.Taxodist()
# use taxodist to calculate similarity of all patients


array([[ 66.06584356,  78.97508709, 103.9778464 ,  62.5700209 ,
         87.73783243,   1.09861229,  22.18652124,  74.38282429,
         88.46765096,  82.52546489, 113.64417365,  86.7792168 ,
         90.6256531 , 112.48052359, 104.17455468, 106.93354045,
         20.36172192,  84.38500525, 110.21719304,  80.61924128,
        103.16439158,  83.40772878, 102.96502372,  48.17046641,
         15.26829466,  20.17940036,   0.        ,  13.26386294,
         94.12279308],
       [ 78.97508709,  43.94386084,  78.18012256,  72.48419584,
         61.71656086,   1.09861229,  27.91691197,  82.15342991,
         78.72602576,  81.1535565 ,  73.45466916,  86.55536599,
         90.26893206,  80.94725022,  70.58845383,  83.47152407,
         23.46849621,  92.12189242,  67.12429789,  83.52893794,
         70.66969829,  82.6813098 ,  65.54496333,  55.04529469,
         15.5557267 ,  17.39877947,   0.        ,  19.21793791,
         62.49324646],
       [103.9778464 ,  78.18012256, 100.88100802,  59.8053

In [88]:
dist_matrix = td.calc_set_sim(pancreas_icd_sets,tree,'levels','wu_palmer','bipartite_matching',normalize=False)


In [96]:
# dist_matrix = td.calc_set_sim(pancreas_icd_sets,tree,'levels','nguyen_almubaid','bipartite_matching',normalize=False)
normalized_sims = utils.normalize(dist_matrix)
dist_matrix[28,28]

94.0

In [90]:
normalized_sims_marius = utils.normalize(matrix)

In [91]:
# here we only compare the whole matrices as a 1D flattened vector
# we need to compare them patient wise or 
# in another way that accounts for the fact that the current vectors hold certain values twice
cor = np.corrcoef(normalized_sims.flatten(),normalized_sims_marius.flatten())
cor

array([[1.        , 0.51437626],
       [0.51437626, 1.        ]])

In [92]:
print(normalized_sims[27,27]) #28,27,26,6,7,25,24,17
print(normalized_sims_marius[27,27])

print(normalized_sims[27,27]) #28,27,26,6,7,25,24,17
print(normalized_sims_marius[27,27])

0.07446808510638298
1.0
0.07446808510638298
1.0


In [93]:
dist_matrix

array([[42.        , 14.27633478, 25.03174603, 14.59617605, 31.72539683,
         1.        ,  6.61904762, 16.66746032, 20.85642136, 16.45634921,
        22.39062049, 22.05396825, 12.71031746, 16.32936508, 24.54206349,
        16.41269841,  7.57142857, 15.02236652, 23.53412698, 15.55      ,
        26.18174603, 17.91825397, 25.71587302,  7.45238095,  3.28571429,
         5.8547619 ,  0.        ,  5.78571429, 29.71904762],
       [14.27633478, 29.        , 13.95952381, 10.9487013 , 20.68333333,
         1.        ,  4.5969697 , 13.20793651, 14.32633478, 13.96601732,
        16.06904762, 11.13888889,  9.66666667, 12.6468254 , 17.23809524,
        12.0968254 ,  6.03030303,  9.48838384, 18.89300144, 12.71031746,
        17.39458874, 13.16046176, 19.29365079,  5.17474747,  3.06363636,
         7.08333333,  0.        ,  3.58585859, 21.41601732],
       [25.03174603, 13.95952381, 65.        , 15.74617605, 32.50714286,
         0.83333333,  8.09047619, 19.46349206, 19.59444444, 20.05873016,
  